In [5]:
!pip3 install onnxruntime
!pip3 install onnxruntime-tools

Optimize ONNX Model

In [3]:
from onnxruntime.tools.symbolic_shape_infer import SymbolicShapeInference
import onnx

# Load your ONNX model
model_path = "models/LeET.onnx"
model = onnx.load(model_path)

# Perform shape inference
inferred_model = SymbolicShapeInference.infer_shapes(model)

# Save the optimized model
optimized_path = "models/optimized/LeET.onnx"
onnx.save(inferred_model, optimized_path)
print(f"Optimized model saved to {optimized_path}")


Optimized model saved to models/optimized/LeET.onnx


Quantization model

In [9]:
from onnxruntime.quantization import quantize_dynamic, QuantType

# Paths to input and output models
model_input = "models/optimized/LeDro.onnx"
model_output = "models/quantization/LeDro.onnx"

# Perform dynamic quantization
quantize_dynamic(
    model_input,  # Input model path
    model_output, # Output model path
    weight_type=QuantType.QInt8,  # Quantization type (INT8)
    per_channel=True  # Enable per-channel quantization
)

print(f"Quantized model saved to {model_output}")


Quantized model saved to models/quantization/LeDro.onnx


In [ ]:
import onnx
model = onnx.load("models/quantization/LeDro.onnx")
print(model.graph.input)

Test Model

In [11]:
import onnxruntime as ort
import numpy as np

# Load model
session = ort.InferenceSession("models/optimized/LeDro.onnx")

# สร้าง dummy input tensor ขนาด [1, 3, 640, 640]
dummy_input = np.random.rand(1, 3, 640, 640).astype(np.float32)

# Run inference
outputs = session.run(None, {"images": dummy_input})

# ดูผลลัพธ์
print(outputs[0].shape)  # Output shape ควรเป็น [1, 6, N]
print(outputs[0])

(1, 6, 8400)
[[[8.48427773e+00 1.59541931e+01 2.13065720e+01 ... 4.65950684e+02
   4.87500702e+02 5.25638306e+02]
  [1.02783031e+01 9.02434444e+00 7.92623711e+00 ... 6.10531067e+02
   6.09212036e+02 6.05621948e+02]
  [2.63193398e+01 4.52514191e+01 5.49507599e+01 ... 3.54533936e+02
   3.14717834e+02 2.34029022e+02]
  [3.25009041e+01 1.91723118e+01 1.58667030e+01 ... 5.94387207e+01
   6.18578491e+01 6.83438721e+01]
  [2.98023224e-08 2.98023224e-08 1.19209290e-07 ... 1.25175714e-03
   7.32481480e-04 1.05947256e-04]
  [8.94069672e-08 8.94069672e-08 5.96046448e-08 ... 1.16261840e-03
   5.92976809e-04 1.31905079e-04]]]
